<a href="https://colab.research.google.com/github/nak650228/ITEC/blob/prework/VideoRestore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#◢ Video Restoration Prework

#◢ 初期設定

In [ ]:
#@title 割り当てられたGPUの確認
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla P100-PCIE-16GB, 460.32.03, 16280 MiB


In [ ]:
#@title ライブラリ等のインストール
%cd /content
!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python
#!pip install torchvision==0.5
!pip install torchvision
#!pip install torch==1.4
!pip install torch==1.9.0
#!pip install scipy==1.2.0
!pip install scipy
#!pip install imgaug==0.2.5
!pip install imgaug
#!pip install tensorflow==1.15.5
!pip install tensorflow

import tensorflow as tf
import youtube_dl
import ffmpeg
import numpy as np
import imageio
import cv2
import torch
import glob
import shutil
import moviepy.editor as mpy
import os
from IPython.display import clear_output
torch.backends.cudnn.benchmark=True

clear_output()


In [ ]:
# Clone realESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Clone BSRGAN
!git clone https://github.com/cszn/BSRGAN.git

# Clone SwinIR
!git clone https://github.com/JingyunLiang/SwinIR.git
!pip install timm

# Download the pre-trained models
!wget https://github.com/cszn/KAIR/releases/download/v1.0/BSRGAN.pth -P BSRGAN/model_zoo
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth -P experiments/pretrained_models

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (390/390), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 390 (delta 208), reused 279 (delta 110), pack-reused 0
Receiving objects: 100% (390/390), 3.20 MiB | 23.75 MiB/s, done.
Resolving deltas: 100% (208/208), done.
/content/Real-ESRGAN
     |████████████████████████████████| 147 kB 7.0 MB/s 
     |████████████████████████████████| 5.8 MB 18.0 MB/s 
     |████████████████████████████████| 185 kB 92.8 MB/s 
  Created wheel for basicsr: filename=basicsr-1.3.4.2-py3-none-any.whl size=182171 sha256=cf14be16eaaee59250f969a07a6d50192e18f99c0b59d03a8efdc951b6640d66
  Stored in directory: /root/.cache/pip/wheels/96/be/05/7e5677c7d34ee56a2a1e2f05e1ce26a0bf358c1e2fb5ee46c8
Successfully built basicsr


     |████████████████████████████████| 1.1 MB 8.0 MB/s 
     |████████████████████████████████| 177 kB 88.0 MB/s 
  Created wheel for facexlib: filename=facexlib-0.2.1.0-py3-none-any.whl size=57093 sha256=19dd0f43abd413c3483b559c8f80e9ddccbb57b99c000698f9ca2640f7a8a753
  Stored in directory: /root/.cache/pip/wheels/e3/33/8d/70d77abe7eca95c8b0f32f12867234c45c1089db9db7102d03
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=30c9879d87698df4121a97579f9f7ac81443cbf41a22d77f839a7bb79874fef3
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built facexlib filterpy
  Created wheel for gfpgan: filename=gfpgan-0.2.1-py3-none-any.whl size=35468 sha256=1e1bda225bc82b19164b61ec0db916242c1f44eea6e27706625ffc1770d45f92
  Stored in directory: /root/.cache/pip/wheels/77/3e/9d/c3334b221f11764db98224dc386fb04d9de005bd7649e37ba0
Successfully built gfpgan
running develop
running egg_info
creatin

In [ ]:
#@title DeepDeblurのモデルをダウンロードしインストールする

%cd /content/
!git clone https://github.com/styler00dollar/Colab-DeblurGANv2
!pip install fire
!pip install pretrainedmodels
!pip install gdown
%cd /content/Colab-DeblurGANv2/models
!gdown --id 1UXcsRVW-6KF23_TNzxw-xC0SzaMfXOaR
!wget --no-check-certificate http://sceneparsing.csail.mit.edu/model/pretrained_resnet/mobilenet_v2.pth.tar
!gdown --id 1JhnT4BBeKBBSLqTo6UsJ13HeBXevarrU

/content
Cloning into 'Colab-DeblurGANv2'...
remote: Enumerating objects: 835, done.
remote: Total 835 (delta 0), reused 0 (delta 0), pack-reused 835
Receiving objects: 100% (835/835), 61.79 MiB | 28.83 MiB/s, done.
Resolving deltas: 100% (438/438), done.
     |████████████████████████████████| 87 kB 4.9 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=38691ef2fd09cada094933300a39448948e91fe9462792bf3dcdd909e1679910
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
     |████████████████████████████████| 58 kB 4.3 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=13c187f2544bda49bb93f84a69e9c243f64728f4629441824806b218fca92a9b
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels
/content/Colab-DeblurGANv2/models
Dow

In [ ]:
#@title DeepDeblur の config.yamlを変更
%%writefile /content/Colab-DeblurGANv2/config/config.yaml
---
project: deblur_gan
experiment_desc: fpn

train:
  files_a: &FILES_A /datasets/my_dataset/**/*.jpg
  files_b: *FILES_A
  size: &SIZE 256
  crop: random
  preload: &PRELOAD false
  preload_size: &PRELOAD_SIZE 0
  bounds: [0, .9]
  scope: geometric
  corrupt: &CORRUPT
    - name: cutout
      prob: 0.5
      num_holes: 3
      max_h_size: 25
      max_w_size: 25
    - name: jpeg
      quality_lower: 70
      quality_upper: 90
    - name: motion_blur
    - name: median_blur
    - name: gamma
    - name: rgb_shift
    - name: hsv_shift
    - name: sharpen

val:
  files_a: *FILES_A
  files_b: *FILES_A
  size: *SIZE
  scope: geometric
  crop: center
  preload: *PRELOAD
  preload_size: *PRELOAD_SIZE
  bounds: [.9, 1]
  corrupt: *CORRUPT

phase: train
warmup_num: 3
model:
  g_name: fpn_mobilenet
  blocks: 9
  d_name: double_gan # may be no_gan, patch_gan, double_gan, multi_scale
  d_layers: 3
  content_loss: perceptual
  adv_lambda: 0.001
  disc_loss: wgan-gp
  learn_residual: True
  norm_layer: instance
  dropout: True

num_epochs: 200
train_batches_per_epoch: 1000
val_batches_per_epoch: 100
batch_size: 1
image_size: [256, 256]

optimizer:
  name: adam
  lr: 0.0001
scheduler:
  name: linear
  start_epoch: 50
  min_lr: 0.0000001

Overwriting /content/Colab-DeblurGANv2/config/config.yaml


In [ ]:
#@title DeepDeblurのpredict.pyを修正
%%writefile /content/Colab-DeblurGANv2/predict.py
import os
from glob import glob
from typing import Optional

import cv2
import numpy as np
import torch
import yaml
from fire import Fire
from tqdm import tqdm

from aug import get_normalize
from models.networks import get_generator


class Predictor:
    def __init__(self, weights_path: str, model_name: str = ''):
        with open('config/config.yaml') as cfg:
            config = yaml.load(cfg)
        model = get_generator(model_name or config['model'])
        model.load_state_dict(torch.load(weights_path)['model'])
        self.model = model.cuda()
        self.model.train(True)
        # GAN inference should be in train mode to use actual stats in norm layers,
        # it's not a bug
        self.normalize_fn = get_normalize()

    @staticmethod
    def _array_to_batch(x):
        x = np.transpose(x, (2, 0, 1))
        x = np.expand_dims(x, 0)
        return torch.from_numpy(x)

    def _preprocess(self, x: np.ndarray, mask: Optional[np.ndarray]):
        x, _ = self.normalize_fn(x, x)
        if mask is None:
            mask = np.ones_like(x, dtype=np.float32)
        else:
            mask = np.round(mask.astype('float32') / 255)

        h, w, _ = x.shape
        block_size = 32
        min_height = (h // block_size + 1) * block_size
        min_width = (w // block_size + 1) * block_size

        pad_params = {'mode': 'constant',
                      'constant_values': 0,
                      'pad_width': ((0, min_height - h), (0, min_width - w), (0, 0))
                      }
        x = np.pad(x, **pad_params)
        mask = np.pad(mask, **pad_params)

        return map(self._array_to_batch, (x, mask)), h, w

    @staticmethod
    def _postprocess(x: torch.Tensor) -> np.ndarray:
        x, = x
        x = x.detach().cpu().float().numpy()
        x = (np.transpose(x, (1, 2, 0)) + 1) / 2.0 * 255.0
        return x.astype('uint8')

    def __call__(self, img: np.ndarray, mask: Optional[np.ndarray], ignore_mask=True) -> np.ndarray:
        (img, mask), h, w = self._preprocess(img, mask)
        with torch.no_grad():
            inputs = [img.cuda()]
            if not ignore_mask:
                inputs += [mask]
            pred = self.model(*inputs)
        return self._postprocess(pred)[:h, :w, :]

def process_video(pairs, predictor, output_dir):
    for video_filepath, mask in tqdm(pairs):
        video_filename = os.path.basename(video_filepath)
        output_filepath = os.path.join(output_dir, os.path.splitext(video_filename)[0]+'_deblur.mp4')
        video_in = cv2.VideoCapture(video_filepath)
        fps = video_in.get(cv2.CAP_PROP_FPS)
        width = int(video_in.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video_in.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frame_num = int(video_in.get(cv2.CAP_PROP_FRAME_COUNT))
        video_out = cv2.VideoWriter(output_filepath, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))
        tqdm.write(f'process {video_filepath} to {output_filepath}, {fps}fps, resolution: {width}x{height}')
        for frame_num in tqdm(range(total_frame_num), desc=video_filename):
            res, img = video_in.read()
            if not res:
                break
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            pred = predictor(img, mask)
            pred = cv2.cvtColor(pred, cv2.COLOR_RGB2BGR)
            video_out.write(pred)

def main(img_pattern: str,
         mask_pattern: Optional[str] = None,
         weights_path='/content/Colab-DeblurGANv2/models/fpn_mobilenet.h5',
         out_dir='submit/',
         side_by_side: bool = False,
         video: bool = False):
    def sorted_glob(pattern):
        return sorted(glob(pattern))

    imgs = sorted_glob(img_pattern)
    masks = sorted_glob(mask_pattern) if mask_pattern is not None else [None for _ in imgs]
    pairs = zip(imgs, masks)
    names = sorted([os.path.basename(x) for x in glob(img_pattern)])
    predictor = Predictor(weights_path=weights_path)

    os.makedirs(out_dir, exist_ok=True)
    if not video:
        for name, pair in tqdm(zip(names, pairs), total=len(names)):
            f_img, f_mask = pair
            img, mask = map(cv2.imread, (f_img, f_mask))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            pred = predictor(img, mask)
            if side_by_side:
                pred = np.hstack((img, pred))
            pred = cv2.cvtColor(pred, cv2.COLOR_RGB2BGR)
            cv2.imwrite(os.path.join(out_dir, name),
                        pred)
    else:
        process_video(pairs, predictor, out_dir)


if __name__ == '__main__':
    Fire(main)

Overwriting /content/Colab-DeblurGANv2/predict.py


In [ ]:
#@title DeepDeblurのfpn_mobilenet.pyを修正
%%writefile /content/Colab-DeblurGANv2/models/fpn_mobilenet.py
import torch
import torch.nn as nn
from models.mobilenet_v2 import MobileNetV2

class FPNHead(nn.Module):
    def __init__(self, num_in, num_mid, num_out):
        super().__init__()

        self.block0 = nn.Conv2d(num_in, num_mid, kernel_size=3, padding=1, bias=False)
        self.block1 = nn.Conv2d(num_mid, num_out, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        x = nn.functional.relu(self.block0(x), inplace=True)
        x = nn.functional.relu(self.block1(x), inplace=True)
        return x


class FPNMobileNet(nn.Module):

    def __init__(self, norm_layer, output_ch=3, num_filters=64, num_filters_fpn=128, pretrained=True):
        super().__init__()

        # Feature Pyramid Network (FPN) with four feature maps of resolutions
        # 1/4, 1/8, 1/16, 1/32 and `num_filters` filters for all feature maps.

        self.fpn = FPN(num_filters=num_filters_fpn, norm_layer = norm_layer, pretrained=pretrained)

        # The segmentation heads on top of the FPN

        self.head1 = FPNHead(num_filters_fpn, num_filters, num_filters)
        self.head2 = FPNHead(num_filters_fpn, num_filters, num_filters)
        self.head3 = FPNHead(num_filters_fpn, num_filters, num_filters)
        self.head4 = FPNHead(num_filters_fpn, num_filters, num_filters)

        self.smooth = nn.Sequential(
            nn.Conv2d(4 * num_filters, num_filters, kernel_size=3, padding=1),
            norm_layer(num_filters),
            nn.ReLU(),
        )

        self.smooth2 = nn.Sequential(
            nn.Conv2d(num_filters, num_filters // 2, kernel_size=3, padding=1),
            norm_layer(num_filters // 2),
            nn.ReLU(),
        )

        self.final = nn.Conv2d(num_filters // 2, output_ch, kernel_size=3, padding=1)

    def unfreeze(self):
        self.fpn.unfreeze()

    def forward(self, x):

        map0, map1, map2, map3, map4 = self.fpn(x)

        map4 = nn.functional.upsample(self.head4(map4), scale_factor=8, mode="nearest")
        map3 = nn.functional.upsample(self.head3(map3), scale_factor=4, mode="nearest")
        map2 = nn.functional.upsample(self.head2(map2), scale_factor=2, mode="nearest")
        map1 = nn.functional.upsample(self.head1(map1), scale_factor=1, mode="nearest")

        smoothed = self.smooth(torch.cat([map4, map3, map2, map1], dim=1))
        smoothed = nn.functional.upsample(smoothed, scale_factor=2, mode="nearest")
        smoothed = self.smooth2(smoothed + map0)
        smoothed = nn.functional.upsample(smoothed, scale_factor=2, mode="nearest")

        final = self.final(smoothed)
        res = torch.tanh(final) + x

        return torch.clamp(res, min=-1, max=1)


class FPN(nn.Module):

    def __init__(self, norm_layer, num_filters=128, pretrained=True):
        """Creates an `FPN` instance for feature extraction.
        Args:
          num_filters: the number of filters in each output pyramid level
          pretrained: use ImageNet pre-trained backbone feature extractor
        """

        super().__init__()
        net = MobileNetV2(n_class=1000)

        if pretrained:
            #Load weights into the project directory
            state_dict = torch.load('/content/Colab-DeblurGANv2/models/mobilenet_v2.pth.tar') # add map_location='cpu' if no gpu
            net.load_state_dict(state_dict)
        self.features = net.features

        self.enc0 = nn.Sequential(*self.features[0:2])
        self.enc1 = nn.Sequential(*self.features[2:4])
        self.enc2 = nn.Sequential(*self.features[4:7])
        self.enc3 = nn.Sequential(*self.features[7:11])
        self.enc4 = nn.Sequential(*self.features[11:16])

        self.td1 = nn.Sequential(nn.Conv2d(num_filters, num_filters, kernel_size=3, padding=1),
                                 norm_layer(num_filters),
                                 nn.ReLU(inplace=True))
        self.td2 = nn.Sequential(nn.Conv2d(num_filters, num_filters, kernel_size=3, padding=1),
                                 norm_layer(num_filters),
                                 nn.ReLU(inplace=True))
        self.td3 = nn.Sequential(nn.Conv2d(num_filters, num_filters, kernel_size=3, padding=1),
                                 norm_layer(num_filters),
                                 nn.ReLU(inplace=True))

        self.lateral4 = nn.Conv2d(160, num_filters, kernel_size=1, bias=False)
        self.lateral3 = nn.Conv2d(64, num_filters, kernel_size=1, bias=False)
        self.lateral2 = nn.Conv2d(32, num_filters, kernel_size=1, bias=False)
        self.lateral1 = nn.Conv2d(24, num_filters, kernel_size=1, bias=False)
        self.lateral0 = nn.Conv2d(16, num_filters // 2, kernel_size=1, bias=False)

        for param in self.features.parameters():
            param.requires_grad = False

    def unfreeze(self):
        for param in self.features.parameters():
            param.requires_grad = True


    def forward(self, x):

        # Bottom-up pathway, from ResNet
        enc0 = self.enc0(x)

        enc1 = self.enc1(enc0) # 256

        enc2 = self.enc2(enc1) # 512

        enc3 = self.enc3(enc2) # 1024

        enc4 = self.enc4(enc3) # 2048

        # Lateral connections

        lateral4 = self.lateral4(enc4)
        lateral3 = self.lateral3(enc3)
        lateral2 = self.lateral2(enc2)
        lateral1 = self.lateral1(enc1)
        lateral0 = self.lateral0(enc0)

        # Top-down pathway
        map4 = lateral4
        map3 = self.td1(lateral3 + nn.functional.upsample(map4, scale_factor=2, mode="nearest"))
        map2 = self.td2(lateral2 + nn.functional.upsample(map3, scale_factor=2, mode="nearest"))
        map1 = self.td3(lateral1 + nn.functional.upsample(map2, scale_factor=2, mode="nearest"))
        return lateral0, map1, map2, map3, map4



Overwriting /content/Colab-DeblurGANv2/models/fpn_mobilenet.py


In [ ]:
#@title DeOldifyの初期設定
%cd /content

!git clone https://github.com/jantic/DeOldify.git DeOldify
%cd DeOldify

#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

from os import path

!pip install -r colab_requirements.txt

import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth

colorizer = get_video_colorizer()

/content
Cloning into 'DeOldify'...
remote: Enumerating objects: 2288, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 2288 (delta 27), reused 17 (delta 5), pack-reused 2228
Receiving objects: 100% (2288/2288), 69.44 MiB | 35.18 MiB/s, done.
Resolving deltas: 100% (1035/1035), done.
/content/DeOldify
     |████████████████████████████████| 214 kB 8.3 MB/s 
     |████████████████████████████████| 129 kB 82.9 MB/s 
     |████████████████████████████████| 78 kB 9.5 MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=5a84b89c804e84612f59f54878c6b9b4006b7e5993626aa53e50da2851bfd126
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built typing
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


--2021-09-26 02:16:52--  https://data.deepai.org/deoldify/ColorizeVideo_gen.pth
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874066230 (834M) [application/octet-stream]
Saving to: ‘./models/ColorizeVideo_gen.pth’

./models/ColorizeVi 100%[===================>] 833.57M  20.0MB/s    in 45s     

2021-09-26 02:17:38 (18.7 MB/s) - ‘./models/ColorizeVideo_gen.pth’ saved [874066230/874066230]



Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [ ]:
#@title ##**Clone the repository of DeepRemaster** { display-mode: "form" }
%cd /content
!git clone https://github.com/satoshiiizuka/siggraphasia2019_remastering.git DeepRemaster
!cp -r /content/video.mp4 /content/DeepRemaster/
%cd /content/DeepRemaster

!wget --continue -O model/remasternet.pth.tar -- http://iizuka.cs.tsukuba.ac.jp/data/remasternet.pth.tar


/content
Cloning into 'DeepRemaster'...
remote: Enumerating objects: 29, done.
remote: Total 29 (delta 0), reused 0 (delta 0), pack-reused 29
Unpacking objects: 100% (29/29), done.
cp: cannot stat '/content/video.mp4': No such file or directory
/content/DeepRemaster
--2021-09-26 02:18:04--  http://iizuka.cs.tsukuba.ac.jp/data/remasternet.pth.tar
Resolving iizuka.cs.tsukuba.ac.jp (iizuka.cs.tsukuba.ac.jp)... 130.158.43.156
Connecting to iizuka.cs.tsukuba.ac.jp (iizuka.cs.tsukuba.ac.jp)|130.158.43.156|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256796033 (245M) [application/x-tar]
Saving to: ‘model/remasternet.pth.tar’

model/remasternet.p 100%[===================>] 244.90M  30.6MB/s    in 8.7s    

2021-09-26 02:18:13 (28.0 MB/s) - ‘model/remasternet.pth.tar’ saved [256796033/256796033]



In [ ]:
#@title Microsoft Bringing-Old-Photos-Back-to-Lifeのリポジトリーをクローンする
%cd /content
!git clone https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life.git photo_restoration

/content
Cloning into 'photo_restoration'...
remote: Enumerating objects: 456, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 456 (delta 72), reused 102 (delta 53), pack-reused 314
Receiving objects: 100% (456/456), 32.25 MiB | 29.41 MiB/s, done.
Resolving deltas: 100% (188/188), done.


In [ ]:
#@title 学習済みモデルをダウンロードダウンロード
# pull the syncBN repo
%cd photo_restoration/Face_Enhancement/models/networks
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../../

%cd Global/detection_models
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../

# download the landmark detection model
%cd Face_Detection/
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
%cd ../

# download the pretrained model
%cd Face_Enhancement/
!wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Face_Enhancement/checkpoints.zip
!unzip -o checkpoints.zip
%cd ../

%cd Global/
!wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Global/checkpoints.zip
!unzip -o checkpoints.zip
%cd ../

! pip install -r requirements.txt

/content/photo_restoration/Face_Enhancement/models/networks
Cloning into 'Synchronized-BatchNorm-PyTorch'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 188 (delta 10), reused 27 (delta 10), pack-reused 161
Receiving objects: 100% (188/188), 47.20 KiB | 878.00 KiB/s, done.
Resolving deltas: 100% (106/106), done.
/content/photo_restoration
/content/photo_restoration/Global/detection_models
Cloning into 'Synchronized-BatchNorm-PyTorch'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 188 (delta 10), reused 27 (delta 10), pack-reused 161
Receiving objects: 100% (188/188), 47.20 KiB | 6.74 MiB/s, done.
Resolving deltas: 100% (106/106), done.
/content/photo_restoration
/content/photo_restoration/Face_Detection
--2021-09-26 02:18:16--  http://dlib.net/files/shape_predictor_68_face_landma

#◢ 画像のダウンロード

In [ ]:
#@title **Googleドライブの追加**
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print('Google Drive connected.')

Mounted at /content/drive
Google Drive connected.


In [ ]:
#@title ##**ビデオのダウンロード** { display-mode: "form" }
#@markdown *ビデオへのリンク（YouTubeやTwitterなど）を入力するか、source_urlフィールドを空白にしてください（空白にした場合、コンピューターからビデオをアップロードするよう求められます）。*
projectname = 'Zombie' #@param {type:"string"}

source_url = 'https://www.youtube.com/watch?v=MQ8ZKw7YIfQ' #@param {type:"string"}

%cd /content
! rm -f /content/*.mp4

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4



fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
width_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_WIDTH))
height_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_HEIGHT))
print ("FPS of VIDEO: ",fps_of_video)
print ("Frames of VIDEO: ",frames_of_video)
print ("Width of VIDEO: ",width_of_video)
print ("Height of VIDEO: ",height_of_video)



/content
[youtube] MQ8ZKw7YIfQ: Downloading webpage
[download] Destination: downloaded_video.f136.mp4
[download] 100% of 424.24MiB in 00:18
[download] Destination: downloaded_video.mp4.f140
[download] 100% of 86.92MiB in 00:03
[ffmpeg] Merging formats into "downloaded_video.mp4"
Deleting original file downloaded_video.f136.mp4 (pass -k to keep)
Deleting original file downloaded_video.mp4.f140 (pass -k to keep)
FPS of VIDEO:  29
Frames of VIDEO:  171977
Width of VIDEO:  960
Height of VIDEO:  720


In [ ]:
#@title ##**ダウンロードした動画を表示** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/downloaded_video.mp4", autoplay=1, maxduration=6000))
else:
  files.download('/content/downloaded_video.mp4')

In [ ]:
#@markdown *1分以上の動画をダウンロードすることはお勧めできません。また、タイトルに「スペース」や「ドット」が含まれている動画はアップロードしないでください。*

#@markdown *実行中にエラーが発生した場合は、このブロックを再度実行します。*

#@title ##**動画の調整** { display-mode: "form" }
#@markdown *動画の長さを変更することができます。*

#@markdown **動画の長さを変更する場合は、その開始時間と終了時間を指定して下さい。**
target_start = '00:06:23' #@param {type:"string"}
target_end = '00:06:25' #@param {type:"string"}

#@markdown **動画の解像度を変更する場合は、新しい解像度を指定してください（例 640 x 480）。この値を指定しない場合は、元の解像度がそのまま使われます。**
width = "" #@param {type:"string"}
height = "" #@param {type:"string"}

rescale = ""
if width != '' and height != '':
#  rescale = f"-s {width}x{height}"
  rescale = f"-vf scale={width}:{height}"

!rm -f cropped_video.mp4
!ffmpeg -i downloaded_video.mp4 $rescale -ss $target_start -to $target_end cropped_video.mp4
clear_output()
!rm -f video.mp4
!cp cropped_video.mp4 video.mp4


#clear_output()

In [ ]:
#@title ##**サイズ調整後の動画を表示（修復対象）** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/video.mp4')

#◢ 画像の修復(ヒストグラムの修正）

In [ ]:
#@title ##**ビデオを静止画フレームに分解する** { display-mode: "form" }

upload_folder = "/content/datas"
frame_folder  = "/content/datas/frames"

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)

os.mkdir(upload_folder)
os.mkdir(frame_folder)

os.chdir(frame_folder)

!ffmpeg -i /content/video.mp4 %09d.png

clear_output()

In [ ]:
#import numpy as np
#import cv2

files = os.listdir(frame_folder)

for file in files:
    img = cv2.imread(file,0)

# create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(img)

    cv2.imwrite(file,cl1)

In [ ]:
#@title ##**修復した静止画を動画に復元** { display-mode: "form" }
#!ffmpeg -vsync 0 -hwaccel cuvid -c:v mjpeg_cuvid -framerate 30 -i /content/TecoGAN/results/My_video/*.png -c:v h264_nvenc quaid2.mp4

histgram_file="/content/histgram_video.mp4"

if os.path.isfile(histgram_file) :
    os.remove(histgram_file)

os.chdir(frame_folder)

subprocess.run('/usr/bin/ffmpeg -f image2 -framerate ' + str(fps_of_video) + ' -i /content/datas/frames/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/histgram_video.mp4' , shell=True )
#!ffmpeg -f image2 -framerate fps_of_video -i %09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/histgram_video.mp4
#!rm -f /content/histgram_video.mp4
!cp /content/histgram_video.mp4 /content/video.mp4

#clear_output()

In [ ]:
#@title ##**ヒストグラム調整後の動画を表示** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/video.mp4')

In [ ]:
#@title ##**手振れ除去**
#@markdown 動画の手振れを補正する場合は、以下のチェックを指定します。手振れ除去を行うと、多少輪郭がはっきりします。**
is_deblur = True #@param {type:"boolean"}

# #◢ 手振れ（Blur）を除去する

In [ ]:
#@title ##**ビデオをフレームに分解する** { display-mode: "form" }
%cd /content/Colab-DeblurGANv2/

if is_deblur == True:
  !python predict.py --out_dir=/content/Colab-DeblurGANv2/submit/ --video=True /content/video.mp4
  !mv submit/video_deblur.mp4 /content/deblured_video.mp4
  !cp /content/deblured_video.mp4 /content/video.mp4

clear_output()

In [ ]:
#@title ##**手振れ補正後の動画を表示** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/video.mp4')

# #◢ ノイズ除去(Deep Remaster)

In [ ]:
#@title ##**Remove frame noise** { display-mode: "form" }
%cd /content/DeepRemaster
#height_num=int(height)
subprocess.run('python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim ' +str(height_num) , shell=True )
#!python remaster.py --input /content/video.mp4 --disable_colorization --gpu
clear_output()

/content/DeepRemaster


NameError: ignored

In [ ]:
#@title ##**ノイズ除去結果の表示** { display-mode: "form" }
!rm -rf /content/video.mp4
!cp -r video_out.mp4 /content/video.mp4
!cp -r video_out.mp4 /content/denoise_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/denoise_video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/denoise_video.mp4')

# #◢ モノクロ動画に色を付ける

In [ ]:
#@title ##**Deoldify のレンダリングファクター指定**
#@markdown **モノクロ動画にAIで色を付ける場合、以下のチェックを指定します。**
id_deoldify = True #@param {type:"boolean"}
#@markdown **Deoldifyのレンダリングファクターを指定します。**
render_factor = 36  #@param {type: "slider", min: 5, max: 44}

In [ ]:
#@title Deoldifyによるモノクロ動画のカラー化
is_deoldify = True

if is_deoldify == True:

  %cd /content/DeOldify

  !rm -rf video
  !mkdir -p 'video/source'

  !cp -r /content/video.mp4 video/source/video.mp4
  video_path = colorizer.colorize_from_file_name('video.mp4', render_factor)
  !cp -r video/result/video.mp4 /content/colorized_video.mp4
  !cp -r video/result/video.mp4 /content/video.mp4
  !rm -f  video/result/video.mp4

/content/DeOldify


Video created here: video/result/video.mp4


In [ ]:
#@title ##**カラー変換後の動画を表示** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/video.mp4')

In [ ]:
################# Required Configurations ############################

##@markdown # Required Configuration
##@markdown Use the values in here to configure what you'd like DAIN to do.

##@markdown ## Input file
##@markdown Path (relative to the root of your Google Drive) to the input file. For instance, if you save your `example.mkv` file in your Google Drive, inside a `videos` folder, the path would be: `videos/example.mkv`. Currenly videos and gifs are supported.
#INPUT_FILEPATH = "/content/video.mp4" #@param{type:"string"}
INPUT_FILEPATH = "/content/video.mp4"

##@markdown ## Output file
##@markdown Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is recommended for video input, `.gif` for gif inputs.
#OUTPUT_FILE_PATH = "/content/beautified_video.mp4" #@param{type:"string"}
OUTPUT_FILE_PATH = "/content/beautified_video.mp4"

################# Optional configurations ############################

#@markdown # Optional Configuration
#@markdown Parameters below can be left with their defaults, but feel free to adapt them to your needs.

#@markdown ## Target FPS
#@markdown  how many frames per second should the result have. This will determine how many intermediate images are interpolated.
TARGET_FPS =  60#@param{type:"number"}

##@markdown ## Frame input directory
##@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
#FRAME_INPUT_DIR = '/content/photo_restoration/input_frames' #@param{type:"string"}
FRAME_INPUT_DIR = '/content/photo_restoration/input_frames'

##@markdown ## Frame output directory
##@markdown A path, relative to your GDrive root, where you want the generated frame.
#FRAME_OUTPUT_DIR = '/content/photo_restoration/output_frames' #@param{type:"string"}
FRAME_OUTPUT_DIR = '/content/photo_restoration/output_frames' 

#@markdown ## Seamless playback
#@markdown Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS = False #@param{type:"boolean"}

#@markdown ## Resize hotfix
#@markdown DAIN frames are a bit "shifted / smaller" compared to original input frames. This can partly be mitigated with resizing DAIN frames to the resolution +2px and cropping the result to the original resoultion with the starting point (1,1). Without this fix, DAIN tends to make "vibrating" output and it is pretty noticible with static elements like text.
#@markdown
#@markdown This hotfix tries to make such effects less visible for a smoother video playback. I do not know what DAINAPP uses as a fix for this problem, but the original does show such behaviour with the default test images. More advanced users can change the interpolation method. The methods cv2.INTER_CUBIC and cv2.INTER_LANCZOS4 are recommended. The current default value is cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True #@param{type:"boolean"}

#@markdown ## Auto-remove PNG directory
#@markdown Auto-delete output PNG dir after ffmpeg video creation. Set this to `False` if you want to keep the PNG files.
AUTO_REMOVE = True #@param{type:"boolean"}

clear_output()

#◢ Microsoft Bringing-Old-Photos-Back-to-Lifeによる画像修正

In [ ]:
#@title 動画ファイルを画像ファイルに分解
# ffmpeg extract - Generating individual frame PNGs from the source file.

%cd /content/photo_restoration
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

%shell ffmpeg -i '{INPUT_FILEPATH}' '{FRAME_INPUT_DIR}/%05d.png'

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
from IPython.display import clear_output

pngs_generated_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS==True:
  pngs_generated_count += 1
  original = str(FRAME_INPUT_DIR)+"/00001.png"
  target = str(FRAME_INPUT_DIR)+"/"+str(pngs_generated_count).zfill(5)+".png"
  shutil.copyfile(original, target)

#print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

# Checking if PNG do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

clear_output()

In [ ]:
#@title 精彩化の実行
%cd /content/photo_restoration
input_folder = FRAME_INPUT_DIR
output_folder = FRAME_OUTPUT_DIR

!rm -rf /content/photo_restoration/output_frames/*

print (input_folder)
print (output_folder)

import os
basepath = os.getcwd()
#input_path = os.path.join(basepath, input_folder)
#output_path = os.path.join(basepath, output_folder)
#os.mkdir(output_path)
#!rm -rf output_folder
#os.mkdir(output_folder)

!python run.py --input_folder /content/photo_restoration/input_frames --output_folder /content/photo_restoration/output_frames --GPU 0

clear_output()

In [ ]:
y#@title ビデオファイルの作成
#create video
%cd /content/photo_restoration/output_frames/final_output
!ffmpeg  -pattern_type glob -i '*.png' -c:v h264_nvenc -pix_fmt yuv420p /content/beautified_video.mp4
!cp /content/output.mp4 /content/drive/MyDrive/Movie

clear_output()

In [ ]:
#@title ##**Get result** { display-mode: "form" }
!rm -rf /content/video.mp4
!cp -r /content/beautified_video.mp4 /content/video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/beautified_video.mp4", autoplay=1,  maxduration=600))
else:
  files.download('/content/beautified_video.mp4')

In [ ]:
#@title DeOldifyの初期設定
%cd /content

!git clone https://github.com/jantic/DeOldify.git DeOldify
%cd DeOldify

#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

from os import path

!pip install -r colab_requirements.txt

import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth

colorizer = get_video_colorizer()

/content
Cloning into 'DeOldify'...
remote: Enumerating objects: 2288, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 2288 (delta 27), reused 17 (delta 5), pack-reused 2228
Receiving objects: 100% (2288/2288), 69.44 MiB | 36.21 MiB/s, done.
Resolving deltas: 100% (1034/1034), done.
/content/DeOldify
     |████████████████████████████████| 214 kB 5.2 MB/s 
     |████████████████████████████████| 129 kB 57.3 MB/s 
     |████████████████████████████████| 78 kB 6.9 MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=41cef0808cf31f44e8ba18e59dc76f8c7b15e41b051a8edab3aad85926f34aab
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built typing
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


--2021-09-24 12:29:56--  https://data.deepai.org/deoldify/ColorizeVideo_gen.pth
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874066230 (834M) [application/octet-stream]
Saving to: ‘./models/ColorizeVideo_gen.pth’

./models/ColorizeVi 100%[===================>] 833.57M  29.1MB/s    in 30s     

2021-09-24 12:30:27 (27.9 MB/s) - ‘./models/ColorizeVideo_gen.pth’ saved [874066230/874066230]



Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

#◢ SwinIRによるアップスケーリング

In [ ]:
#@title ##**ビデオを静止画フレームに分解する** { display-mode: "form" }

frame_folder  = "/content/Real-ESRGAN/BSRGAN/testsets/RealSRSet"

if os.path.isdir(frame_folder):
    shutil.rmtree(frame_folder)

os.mkdir(frame_folder)

os.chdir(frame_folder)

!ffmpeg -i /content/video.mp4 %09d.png

clear_output()

In [ ]:
# SwinIR
os.chdir("/content/Real-ESRGAN")
!python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth --folder_lq BSRGAN/testsets/RealSRSet --scale 4
shutil.move('results/swinir_real_sr_x4', 'results/SwinIR')

loading model from experiments/pretrained_models/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth
Testing 0 000000001           
Testing 1 000000002           
Testing 2 000000003           
Testing 3 000000004           
Testing 4 000000005           
Testing 5 000000006           
Testing 6 000000007           
Testing 7 000000008           
Testing 8 000000009           
Testing 9 000000010           
Testing 10 000000011           
Testing 11 000000012           
Testing 12 000000013           
Testing 13 000000014           
Testing 14 000000015           
Testing 15 000000016           
Testing 16 000000017           
Testing 17 000000018           
Testing 18 000000019           
Testing 19 000000020           
Testing 20 000000021           
Testing 21 000000022           
Testing 22 000000023           
Testing 23 000000024           
Testing 24 000000025           
Testing 25 000000026           
Testing 26 000000027           
Testing 27 000000028           
Testing 28 000000

'results/SwinIR'

In [ ]:
#@title ##**修復した静止画を動画に復元** { display-mode: "form" }
#!ffmpeg -vsync 0 -hwaccel cuvid -c:v mjpeg_cuvid -framerate 30 -i /content/TecoGAN/results/My_video/*.png -c:v h264_nvenc quaid2.mp4

histgram_file="/content/SRed_video.mp4"
result_folder="/content/Real-ESRGAN/results/SwinIR/"

if os.path.isfile(histgram_file) :
    os.remove(histgram_file)

os.chdir(result_folder)

subprocess.run('/usr/bin/ffmpeg -f image2 -framerate ' + str(fps_of_video) + ' -i /content/datas/frames/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/histgram_video.mp4' , shell=True )
#!ffmpeg -f image2 -framerate fps_of_video -i %09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/histgram_video.mp4
#!rm -f /content/histgram_video.mp4
!cp /content/SRed_video.mp4 /content/video.mp4

#clear_output()

cp: cannot stat '/content/SRed_video.mp4': No such file or directory


#◢ TECOGANによるアップスケーリング

In [ ]:
#@title TECOGANのインストール準備
%cd /content
!git clone https://github.com/thunil/TecoGAN.git

%cd TecoGAN
!pip install -r requirements.txt


/content
fatal: destination path 'TecoGAN' already exists and is not an empty directory.
/content/TecoGAN


In [ ]:
!python runGan.py 0


Testing test case 0
--2021-07-18 06:45:20--  https://ge.in.tum.de/download/data/TecoGAN/model.zip
Resolving ge.in.tum.de (ge.in.tum.de)... 129.187.254.228, 2001:4ca0:0:103::81bb:fee4
Connecting to ge.in.tum.de (ge.in.tum.de)|129.187.254.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11288778 (11M) [application/zip]
Saving to: ‘model/model.zip’

model/model.zip     100%[===================>]  10.77M   593KB/s    in 19s     

2021-07-18 06:45:40 (579 KB/s) - ‘model/model.zip’ saved [11288778/11288778]

Archive:  model/model.zip
replace model/TecoGAN.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: model/TecoGAN.data-00000-of-00001  
  inflating: model/TecoGAN.index     
  inflating: model/TecoGAN.meta      
--2021-07-18 06:45:49--  https://ge.in.tum.de/download/data/TecoGAN/vid3_LR.zip
Resolving ge.in.tum.de (ge.in.tum.de)... 129.187.254.228, 2001:4ca0:0:103::81bb:fee4
Connecting to ge.in.tum.de (ge.in.tum.de)|129.187.254.228|:443..

In [ ]:
!rm -rf /content/TecoGAN/LR/My_video/
!mkdir /content/TecoGAN/LR/My_video
%cd /content/TecoGAN/LR/My_video
!ffmpeg -i /content/video.mp4 %05d.png

%cd /content/TecoGAN/


/content/TecoGAN/LR/My_video
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 

In [ ]:
%%writefile runGan.py
'''
several running examples, run with
python3 runGan.py 1 # the last number is the run case number
runcase == 1    inference a trained model
runcase == 2    calculate the metrics, and save the numbers in csv
runcase == 3    training TecoGAN
runcase == 4    training FRVSR
runcase == ...  coming... data preparation and so on...
'''
import os, subprocess, sys, datetime, signal, shutil
 
runcase = int(sys.argv[1])
print ("Testing test case %d" % runcase)
 
def preexec(): # Don't forward signals.
    os.setpgrp()
    
def mycall(cmd, block=False):
    if not block:
        return subprocess.Popen(cmd)
    else:
        return subprocess.Popen(cmd, preexec_fn = preexec)
    
def folder_check(path):
    try_num = 1
    oripath = path[:-1] if path.endswith('/') else path
    while os.path.exists(path):
        print("Delete existing folder " + path + "?(Y/N)")
        decision = input()
        if decision == "Y":
            shutil.rmtree(path, ignore_errors=True)
            break
        else:
            path = oripath + "_%d/"%try_num
            try_num += 1
            print(path)
    
    return path
 
if( runcase == 0 ): # download inference data, trained models
    # download the trained model
    if(not os.path.exists("./model/")): os.mkdir("./model/")
    cmd1 = "wget https://ge.in.tum.de/download/data/TecoGAN/model.zip -O model/model.zip;"
    cmd1 += "unzip model/model.zip -d model; rm model/model.zip"
    subprocess.call(cmd1, shell=True)
    
    # download some test data
    cmd2 = "wget https://ge.in.tum.de/download/data/TecoGAN/vid3_LR.zip -O LR/vid3.zip;"
    cmd2 += "unzip LR/vid3.zip -d LR; rm LR/vid3.zip"
    subprocess.call(cmd2, shell=True)
    
    cmd2 = "wget https://ge.in.tum.de/download/data/TecoGAN/tos_LR.zip -O LR/tos.zip;"
    cmd2 += "unzip LR/tos.zip -d LR; rm LR/tos.zip"
    subprocess.call(cmd2, shell=True)
    
    # download the ground-truth data
    if(not os.path.exists("./HR/")): os.mkdir("./HR/")
    cmd3 = "wget https://ge.in.tum.de/download/data/TecoGAN/vid4_HR.zip -O HR/vid4.zip;"
    cmd3 += "unzip HR/vid4.zip -d HR; rm HR/vid4.zip"
    subprocess.call(cmd3, shell=True)
    
    cmd3 = "wget https://ge.in.tum.de/download/data/TecoGAN/tos_HR.zip -O HR/tos.zip;"
    cmd3 += "unzip HR/tos.zip -d HR; rm HR/tos.zip"
    subprocess.call(cmd3, shell=True)
    
elif( runcase == 1 ): # inference a trained model
    
    dirstr = '/content/TecoGAN/results/' # the place to save the results
    testpre = ['My_video'] # the test cases
    
    if (not os.path.exists(dirstr)): os.mkdir(dirstr)
    
    # run these test cases one by one:
    for nn in range(len(testpre)):
        cmd1 = ["python3", "main.py",
            "--cudaID", "0",            # set the cudaID here to use only one GPU
            "--output_dir",  dirstr,    # Set the place to put the results.
            "--summary_dir", os.path.join(dirstr, 'log/'), # Set the place to put the log. 
            "--mode","inference", 
            "--input_dir_LR", os.path.join("./LR/", testpre[nn]),   # the LR directory 
            #"--input_dir_HR", os.path.join("./HR/", testpre[nn]),  # the HR directory
            # one of (input_dir_HR,input_dir_LR) should be given
            "--output_pre", testpre[nn], # the subfolder to save current scene, optional
            "--num_resblock", "16",  # our model has 16 residual blocks, 
            # the pre-trained FRVSR and TecoGAN mini have 10 residual blocks
            "--checkpoint", './model/TecoGAN',  # the path of the trained model,
            "--output_ext", "png"               # png is more accurate, jpg is smaller
        ]
        mycall(cmd1).communicate()
 
elif( runcase == 2 ): # calculate all metrics, and save the csv files, should use png
 
    testpre = ["calendar"] # just put more scenes to evaluate all of them
    dirstr = './results/'  # the outputs
    tarstr = './HR/'       # the GT
 
    tar_list = [(tarstr+_) for _ in testpre]
    out_list = [(dirstr+_) for _ in testpre]
    cmd1 = ["python3", "metrics.py",
        "--output", dirstr+"metric_log/",
        "--results", ",".join(out_list),
        "--targets", ",".join(tar_list),
    ]
    mycall(cmd1).communicate()
    
elif( runcase == 3 ): # Train TecoGAN
    '''
    In order to use the VGG as a perceptual loss,
    we download from TensorFlow-Slim image classification model library:
    https://github.com/tensorflow/models/tree/master/research/slim    
    '''
    VGGPath = "model/" # the path for the VGG model, there should be a vgg_19.ckpt inside
    VGGModelPath = os.path.join(VGGPath, "vgg_19.ckpt")
    if(not os.path.exists(VGGPath)): os.mkdir(VGGPath)
    if(not os.path.exists(VGGModelPath)):
        # Download the VGG 19 model from 
        print("VGG model not found, downloading to %s"%VGGPath)
        cmd0 = "wget http://download.tensorflow.org/models/vgg_19_2016_08_28.tar.gz -O " + os.path.join(VGGPath, "vgg19.tar.gz")
        cmd0 += ";tar -xvf " + os.path.join(VGGPath,"vgg19.tar.gz") + " -C " + VGGPath + "; rm "+ os.path.join(VGGPath, "vgg19.tar.gz")
        subprocess.call(cmd0, shell=True)
        
    '''
    Use our pre-trained FRVSR model. If you want to train one, try runcase 4, and update this path by:
    FRVSRModel = "ex_FRVSRmm-dd-hh/model-500000"
    '''
    FRVSRModel = "model/ourFRVSR" 
    if(not os.path.exists(FRVSRModel+".data-00000-of-00001")):
        # Download our pre-trained FRVSR model
        print("pre-trained FRVSR model not found, downloading")
        cmd0 = "wget http://ge.in.tum.de/download/2019-TecoGAN/FRVSR_Ours.zip -O model/ofrvsr.zip;"
        cmd0 += "unzip model/ofrvsr.zip -d model; rm model/ofrvsr.zip"
        subprocess.call(cmd0, shell=True)
    
    TrainingDataPath = "/mnt/netdisk/video_data/" 
    
    '''Prepare Training Folder'''
    # path appendix, manually define it, or use the current datetime, now_str = "mm-dd-hh"
    now_str = datetime.datetime.now().strftime("%m-%d-%H")
    train_dir = folder_check("ex_TecoGAN%s/"%now_str)
    # train TecoGAN, loss = l2 + VGG54 loss + A spatio-temporal Discriminator
    cmd1 = ["python3", "main.py",
        "--cudaID", "0", # set the cudaID here to use only one GPU
        "--output_dir", train_dir, # Set the place to save the models.
        "--summary_dir", os.path.join(train_dir,"log/"), # Set the place to save the log. 
        "--mode","train",
        "--batch_size", "4" , # small, because GPU memory is not big
        "--RNN_N", "10" , # train with a sequence of RNN_N frames, >6 is better, >10 is not necessary
        "--movingFirstFrame", # a data augmentation
        "--random_crop",
        "--crop_size", "32",
        "--learning_rate", "0.00005",
        # -- learning_rate step decay, here it is not used --
        "--decay_step", "500000", 
        "--decay_rate", "1.0", # 1.0 means no decay
        "--stair",
        "--beta", "0.9", # ADAM training parameter beta
        "--max_iter", "500000", # 500k or more, the one we present is trained for 900k
        "--save_freq", "10000", # the frequency we save models
        # -- network architecture parameters --
        "--num_resblock", "16", # FRVSR and TecoGANmini has num_resblock as 10. The TecoGAN has 16.
        # -- VGG loss, disable with vgg_scaling < 0
        "--vgg_scaling", "0.2",
        "--vgg_ckpt", VGGModelPath, # necessary if vgg_scaling > 0
    ]
    '''Video Training data:
    please udate the TrainingDataPath according to ReadMe.md
    input_video_pre is hard coded as scene in dataPrepare.py at line 142
    str_dir is the starting index for training data
    end_dir is the ending index for training data
    end_dir+1 is the starting index for validation data
    end_dir_val is the ending index for validation data
    max_frm should be duration (in dataPrepare.py) -1
    queue_thread: how many cpu can be used for loading data when training
    name_video_queue_capacity, video_queue_capacity: how much memory can be used
    '''
    cmd1 += [
        "--input_video_dir", TrainingDataPath, 
        "--input_video_pre", "scene",
        "--str_dir", "2000",
        "--end_dir", "2250",
        "--end_dir_val", "2290",
        "--max_frm", "119",
        # -- cpu memory for data loading --
        "--queue_thread", "12",# Cpu threads for the data. >4 to speedup the training
        "--name_video_queue_capacity", "1024",
        "--video_queue_capacity", "1024",
    ]
    '''
    loading the pre-trained model from FRVSR can make the training faster
    --checkpoint, path of the model, here our pre-trained FRVSR is given
    --pre_trained_model,  to continue an old (maybe accidentally stopeed) training, 
        pre_trained_model should be false, and checkpoint should be the last model such as 
        ex_TecoGANmm-dd-hh/model-xxxxxxx
        To start a new and different training, pre_trained_model is True. 
        The difference here is 
        whether to load the whole graph icluding ADAM training averages/momentums/ and so on
        or just load existing pre-trained weights.
    '''
    cmd1 += [ # based on a pre-trained FRVSR model. Here we want to train a new adversarial training
        "--pre_trained_model", # True
        "--checkpoint", FRVSRModel,
    ]
    
    # the following can be used to train TecoGAN continuously
    # old_model = "model/ex_TecoGANmm-dd-hh/model-xxxxxxx" 
    # cmd1 += [ # Here we want to train continuously
    #     "--nopre_trained_model", # False
    #     "--checkpoint", old_model,
    # ]
    
    ''' parameters for GAN training '''
    cmd1 += [
        "--ratio", "0.01",  # the ratio for the adversarial loss from the Discriminator to the Generator
        "--Dt_mergeDs",     # if Dt_mergeDs == False, only use temporal inputs, so we have a temporal Discriminator
                            # else, use both temporal and spatial inputs, then we have a Dst, the spatial and temporal Discriminator
    ]
    ''' if the generator is pre-trained, to fade in the discriminator is usually more stable.
    the weight of the adversarial loss will be weighed with a weight, started from Dt_ratio_0, 
    and increases until Dt_ratio_max, the increased value is Dt_ratio_add per training step
    For example, fading Dst in smoothly in the first 4k steps is 
    "--Dt_ratio_max", "1.0", "--Dt_ratio_0", "0.0", "--Dt_ratio_add", "0.00025"
    '''
    cmd1 += [ # here, the fading in is disabled 
        "--Dt_ratio_max", "1.0",
        "--Dt_ratio_0", "1.0", 
        "--Dt_ratio_add", "0.0", 
    ]
    ''' Other Losses '''
    cmd1 += [
        "--pingpang",           # our Ping-Pang loss
        "--pp_scaling", "0.5",  # the weight of the our bi-directional loss, 0.0~0.5
        "--D_LAYERLOSS",        # use feature layer losses from the discriminator
    ]
    
    pid = mycall(cmd1, block=True) 
    try: # catch interruption for training
        pid.communicate()
    except KeyboardInterrupt: # Ctrl + C to stop current training try to save the last model 
        print("runGAN.py: sending SIGINT signal to the sub process...")
        pid.send_signal(signal.SIGINT)
        # try to save the last model 
        pid.communicate()
        print("runGAN.py: finished...")
        
        
elif( runcase == 4 ): # Train FRVSR, loss = l2 warp + l2 content
    now_str = datetime.datetime.now().strftime("%m-%d-%H")
    train_dir = folder_check("ex_FRVSR%s/"%now_str)
    cmd1 = ["python3", "main.py",
        "--cudaID", "0", # set the cudaID here to use only one GPU
        "--output_dir", train_dir, # Set the place to save the models.
        "--summary_dir", os.path.join(train_dir,"log/"), # Set the place to save the log. 
        "--mode","train",
        "--batch_size", "4" , # small, because GPU memory is not big
        "--RNN_N", "10" , # train with a sequence of RNN_N frames, >6 is better, >10 is not necessary
        "--movingFirstFrame", # a data augmentation
        "--random_crop",
        "--crop_size", "32",
        "--learning_rate", "0.00005",
        # -- learning_rate step decay, here it is not used --
        "--decay_step", "500000", 
        "--decay_rate", "1.0", # 1.0 means no decay
        "--stair",
        "--beta", "0.9", # ADAM training parameter beta
        "--max_iter", "500000", # 500k is usually fine for FRVSR, GAN versions need more to be stable
        "--save_freq", "10000", # the frequency we save models
        # -- network architecture parameters --
        "--num_resblock", "10", # a smaller model
        "--ratio", "-0.01",  # the ratio for the adversarial loss, negative means disabled
        "--nopingpang",
    ]
    '''Video Training data... Same as runcase 3...'''
    TrainingDataPath = "/mnt/netdisk/video_data/"
    cmd1 += [
        "--input_video_dir", TrainingDataPath, 
        "--input_video_pre", "scene",
        "--str_dir", "2000",
        "--end_dir", "2250",
        "--end_dir_val", "2290",
        "--max_frm", "119",
        # -- cpu memory for data loading --
        "--queue_thread", "12",# Cpu threads for the data. >4 to speedup the training
        "--name_video_queue_capacity", "1024",
        "--video_queue_capacity", "1024",
    ]
    
    pid = mycall(cmd1, block=True)
    try: # catch interruption for training
        pid.communicate()
    except KeyboardInterrupt: # Ctrl + C to stop current training try to save the last model 
        print("runGAN.py: sending SIGINT signal to the sub process...")
        pid.send_signal(signal.SIGINT)
        # try to save the last model 
        pid.communicate()
        print("runGAN.py: finished...")


Overwriting runGan.py


In [ ]:
%tensorflow_version 1.x
!cp /content/drive/MyDrive/MovieBeautifier/main.py /content/TecoGAN/main.py
!cp /content/drive/MyDrive/MovieBeautifier/frvsr.py /content/TecoGAN/lib/frvsr.py
!cp /content/drive/MyDrive/MovieBeautifier/ops.py /content/TecoGAN/lib/ops.py
!python3 runGan.py 1 --input_dir_LR /content/TecoGAN/LR/ 

Testing test case 1
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Using TensorFlow backend.
input shape: [1, 240, 320, 3]
output shape: [1, 960, 1280, 3]
Finish building the network
Loading weights from ckpt model
Frame evaluation starts!!
Warming up 5
Warming up 4
Warming up 3
Warming up 2
Warming up 1
saving image output_00001
saving image output_00002
saving image output_00003
saving image output_00004
saving image output_00005
saving image output_00006
saving image output_00007
saving image output_00008
saving image output_00009
saving image output_00010
saving image output_00011
saving image output_00012
saving image output_00013
saving image output_00014
saving image 

In [ ]:
#!ffmpeg -vsync 0 -hwaccel cuvid -c:v mjpeg_cuvid -framerate 24.00 -i /content/TecoGAN/results/My_video/*.png -c:v h264_nvenc quaid2.mp4
!ffmpeg -f image2 -framerate 30 -i /content/TecoGAN/results/My_video/output_%05d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/upscaled_video.mp4
!rm -f /content/video.mp4
!cp /content/upscaled_video.mp4 /content/video.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
#@title ##**結果の表示** { display-mode: "form" }
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/upscaled_video.mp4", height=400, autoplay=1, maxduration=600))
else:
  files.download("/content/upscaled_video.mp4")

#◢ 最終処理

In [ ]:
#@title ファイルのバックアップ

ProjectDir="/content/drive/MyDrive/Movie/"+str(projectname)
print("ProjectDir")
os.mkdir(ProjectDir)
os.chdir(ProjectDir)
!mv /content/*.mp4 .

ProjectDir
